# 2.2.3 節
コメントは、勉強会で扱った後に充実させる。

### 準備

リスト用ユーティリティ+$\alpha$

In [3]:
(define nil (list))
(define (push x xs)
  (if (null? xs) (list x)
      (cons (car xs) (push x (cdr xs)))))
(define (leaf? t)
  (not (pair? t)))

(define (show-lines . xs)
  (define (iter xs)
    (cond ((null? xs) #t)
          (else (display (car xs))
                (newline)
                (iter (cdr xs)))))
  (iter xs))
(define (square x) (* x x))

以下は、この節の本文中で定義されたもの。

In [33]:
(define (filter predicate seq)
  (cond ((null? seq) nil)
        ((predicate (car seq))
         (cons (car seq) (filter predicate (cdr seq))))
        (else (filter predicate (cdr seq)))))
(define (accumulate op init seq)
  (if (null? seq) init
      (op (car seq) (accumulate op init (cdr seq)))))
(define (enumerate-interval low high)
  (if (> low high) nil
      (cons low (enumerate-interval (+ low 1) high))))

これらを利用して、後で役立つ手続きを定義しておく。

In [34]:
(define (all predicate seq)
  (accumulate (lambda (x y) (and x y)) #t
              (map predicate seq)))
(define (any predicate seq)
  (accumulate (lambda (x y) (or x y)) #f
              (map predicate seq)))

(define (prime? n)
  (define (prime-to-n? m)
    (not (= (remainder n m) 0)))
  (all prime-to-n? (enumerate-interval 2 (- n 1))))
(prime? 134)

#f

## Ex. 2.33
ここで`map`を上書きしてしまうとEx. 2.37に支障が出るので、名前を変えてある。

In [35]:
(define (map~ p sequence)
  (accumulate (lambda (x y) (cons (p x) y)) nil sequence))
(define (append seq1 seq2)
  (accumulate cons seq2 seq1))
(define (length sequence)
  (accumulate (lambda (x y) (+ 1 y)) 0 sequence))


## Ex. 2.34

In [36]:
(define (horner-eval x coefficient-sequence)
  (accumulate (lambda (this-coeff higher-terms) (+ this-coeff (* higher-terms x)))
              0
              coefficient-sequence))

(horner-eval 2 (list 1 3 0 5 0 1))

79

## Ex. 2.35

In [37]:
(define (count-leaves t)
  (accumulate +
              0
              (map (lambda (sub-tree) (count-leaves subtree)))))

In [38]:
(define (count-leaves t) (length (fringe t)))

## Ex. 2.36

In [39]:
(define (accumulate-n op init seqs)
  (if (null? (car seqs)) nil
      (cons (accumulate op init (map car seqs))
            (accumulate-n op init (map cdr seqs)))))

(accumulate-n + 0 (list (list 1 2 3) (list 4 5 6) (list 7 8 9) (list 10 11 12)))

(22 26 30)

## Ex. 2.37

In [40]:
(define (dot-product v w)
  (accumulate + 0 (map * v w)))

(dot-product (list 1 2) (list 2 3))

8

In [41]:
(define (matrix-*-vector m v)
  (map (lambda (row) (dot-product row v)) m))

In [42]:
(define (transpose mat)
  (accumulate-n cons nil mat))

In [43]:
(define (matrix-*-matrix m n)
  (let ((cols (transpose n)))
    (map (lambda (col) (matrix-*-vector m col)) cols)))

## Ex. 2.38
動作イメージ。
$$\verb+(fold-right * i a)+
= \verb+a+_1 * ( \dots * (\verb+a+_{n-1} * (\verb+a+_n * i)))$$
$$\verb+(fold-left * i a)+
= (((i*\verb+a+_1) * \verb+a+_2) * \dots ) * \verb+a+_n$$

In [44]:
(define (fold-right op init seq) (accumulate op init seq))
(define (fold-left op init seq)
  (define (iter result rest)
    (if (null? rest) result
        (iter (op result (car rest)) (cdr rest))))
  (iter init seq))

$$
1 / (2 / (3 / 1)) = 3/2
$$
$$
((1 / 1) / 2) / 3 = 1/6
$$
後半は分かりやすいので省略。

In [45]:
(show-lines
 (fold-right / 1 (list 1 2 3))
 (fold-left / 1 (list 1 2 3))
 (fold-right list nil (list 1 2 3))
 (fold-left list nil (list 1 2 3)))

3/2
1/6
(1 (2 (3 ())))
(((() 1) 2) 3)


#t

「あらゆる列に対して」と言ったときに、`init`について考えないのであれば、
`op`が結合的であることが、2つの手続きが同じ結果を生み出すために必要十分。
$$\verb+(op (op x y) z)+ = \verb+(op x (op y z))+$$
ただし、両者では`init`が異なる向きから演算されるので、
それによって結果が異なることはありうる。
`init`が単位元であれば問題ないが、
それ以外の場合でも結果が一致することはありうるので、必要十分条件を挙げることは難しい。

## Ex. 2.39

In [46]:
(define (reverse seq)
  (fold-right (lambda (x y) (push x y)) nil seq))
(reverse (list 1 2 3 4))

(4 3 2 1)

In [47]:
(define (reverse seq)
  (fold-left (lambda (x y) (cons y x)) nil seq))
(reverse (list 1 2 3 4))

(4 3 2 1)

In [48]:
(define (flatmap proc seq)
  (accumulate append nil (map proc seq)))

## Ex. 2.40

In [49]:
(define (unique-pairs n)
  (flatmap (lambda (i)
             (map (lambda (j) (list i j)) (enumerate-interval 1 (- i 1))))
           (enumerate-interval 1 n)))

(unique-pairs 4)

((2 1) (3 1) (3 2) (4 1) (4 2) (4 3))

In [50]:
(define (prime-sum? pair)
  (prime? (+ (car pair) (cadr pair))))
(define (make-pair-sum pair)
  (list (car pair) (cadr pair) (+ (car pair) (cadr pair))))
(define (prime-sum-pairs n)
  (map make-pair-sum
       (filter prime-sum?
               (unique-pairs n))))

(prime-sum-pairs 10)

((2 1 3) (3 2 5) (4 1 5) (4 3 7) (5 2 7) (6 1 7) (6 5 11) (7 4 11) (7 6 13) (8 3 11) (8 5 13) (9 2 11) (9 4 13) (9 8 17) (10 1 11) (10 3 13) (10 7 17) (10 9 19))

## Ex. 2.41


In [51]:
(define (unique-triples n)
  (flatmap (lambda (pair)
             (map (lambda (k) (push k pair))
                  (enumerate-interval 1 (- (cadr pair) 1))))
           (unique-pairs n)))

(unique-triples 6)

((3 2 1) (4 2 1) (4 3 1) (4 3 2) (5 2 1) (5 3 1) (5 3 2) (5 4 1) (5 4 2) (5 4 3) (6 2 1) (6 3 1) (6 3 2) (6 4 1) (6 4 2) (6 4 3) (6 5 1) (6 5 2) (6 5 3) (6 5 4))

In [52]:
(define (s-sum-triples s n)
  (define (s-sum? triple)
    (= (+ (car triple) (cadr triple) (caddr triple)) s))
  (filter s-sum? (unique-triples n)))

(s-sum-triples 20 10)

((8 7 5) (9 6 5) (9 7 4) (9 8 3) (10 6 4) (10 7 3) (10 8 2) (10 9 1))

## Ex. 2.42

In [53]:
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
         (flatmap
          (lambda (rest-of-queens)
            (map (lambda (new-row)
                   (adjoin-position
                    new-row k rest-of-queens))
                 (enumerate-interval 1 board-size)))
          (queen-cols (- k 1))))))
  (queen-cols board-size))

(define empty-board nil)
(define (safe? k positions)
  (let ((new-pos (car positions)) (rest-pos (cdr positions)))
    (define (safe-to-new-pos? pos2)
      (let ((x1 (car new-pos)) (y1 (cadr new-pos)) (x2 (car pos2)) (y2 (cadr pos2)))
        (not (or (= x1 x2)
                 (= y1 y2)
                 (= (+ x1 y1) (+ x2 y2))
                 (= (- x1 y1) (- x2 y2))))))
    (all safe-to-new-pos? rest-pos)))

(define (adjoin-position new-row k rest-of-positions)
  (cons (list new-row k) rest-of-positions))

(queens 5)

(((4 5) (2 4) (5 3) (3 2) (1 1)) ((3 5) (5 4) (2 3) (4 2) (1 1)) ((5 5) (3 4) (1 3) (4 2) (2 1)) ((4 5) (1 4) (3 3) (5 2) (2 1)) ((5 5) (2 4) (4 3) (1 2) (3 1)) ((1 5) (4 4) (2 3) (5 2) (3 1)) ((2 5) (5 4) (3 3) (1 2) (4 1)) ((1 5) (3 4) (5 3) (2 2) (4 1)) ((3 5) (1 4) (4 3) (2 2) (5 1)) ((2 5) (4 4) (1 3) (3 2) (5 1)))

In [54]:
(and #f (list #t #f))

#f

## Ex. 2.43

In [55]:
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
         (flatmap
          (lambda (new-row)
            (map (lambda (rest-of-queens)
                   (adjoin-position
                    new-row k rest-of-queens))
                 (queen-cols (- k 1))))
          (enumerate-interval 1 board-size)))))
  (queen-cols board-size))

(define empty-board nil)
(define (safe? k positions)
  (let ((new-pos (car positions)) (rest-pos (cdr positions)))
    (define (safe-to-new-pos? pos2)
      (let ((x1 (car new-pos)) (y1 (cadr new-pos)) (x2 (car pos2)) (y2 (cadr pos2)))
        (not (or (= x1 x2)
                 (= y1 y2)
                 (= (+ x1 y1) (+ x2 y2))
                 (= (- x1 y1) (- x2 y2))))))
    (all safe-to-new-pos? rest-pos)))

(define (adjoin-position new-row k rest-of-positions)
  (cons (list new-row k) rest-of-positions))

(queens 5)

(((1 5) (3 4) (5 3) (2 2) (4 1)) ((1 5) (4 4) (2 3) (5 2) (3 1)) ((2 5) (4 4) (1 3) (3 2) (5 1)) ((2 5) (5 4) (3 3) (1 2) (4 1)) ((3 5) (1 4) (4 3) (2 2) (5 1)) ((3 5) (5 4) (2 3) (4 2) (1 1)) ((4 5) (1 4) (3 3) (5 2) (2 1)) ((4 5) (2 4) (5 3) (3 2) (1 1)) ((5 5) (2 4) (4 3) (1 2) (3 1)) ((5 5) (3 4) (1 3) (4 2) (2 1)))